### References:

- https://pcjericks.github.io/py-gdalogr-cookbook/geometry.html

In [ ]:
import os
import random

import pyspark.sql.functions as F
import sparkgeo.functions as S
from osgeo import ogr, osr

In [ ]:
gdb = ogr.GetDriverByName("OpenFileGDB")

In [ ]:
gdb.GetName()

In [ ]:
gdb_path = "pointm.gdb"
gdb_name = "points"

In [ ]:
if os.path.exists(gdb_path):
    gdb.DeleteDataSource(gdb_path)

In [ ]:
# Create the data source
data_source = gdb.CreateDataSource(gdb_path)

if data_source is None:
    raise Exception(f"Cannot create {gdb_path}.")

# Create the layer
srs = osr.SpatialReference()
srs.ImportFromEPSG(4326)
layer = data_source.CreateLayer(gdb_name, srs, ogr.wkbPointZM)

# Define and create an ID field
id_field = ogr.FieldDefn("ID", ogr.OFTInteger)
layer.CreateField(id_field)

# Generate random points with M values
num_points = 10
x_min, x_max, y_min, y_max = -180, 180, -90, 90
m_min, m_max = 0, 100

for i in range(num_points):
    point = ogr.Geometry(ogr.wkbPointZM)
    x = random.uniform(x_min, x_max)
    y = random.uniform(y_min, y_max)
    m = random.uniform(m_min, m_max)
    point.AddPointZM(x, y, m, 0.0)
    # point.AddPointM(x, y, m)

    feature = ogr.Feature(layer.GetLayerDefn())
    if i < 5:
        feature.SetGeometry(point)
    feature.SetField("ID", i)
    layer.CreateFeature(feature)
    feature.Destroy()
    point.Destroy()

# Close and clean up the data source
data_source.Destroy()

In [ ]:
df = spark.read.format("gdb").options(path=gdb_path, name=gdb_name).load()

In [ ]:
df.printSchema()

In [ ]:
df.select(
    "ID",
    "Shape.x",
    "Shape.y",
    "Shape.z",
    "Shape.m",    
).show(truncate=False, vertical=True)